<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myProspects_Upside_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance==0.2.59
!pip install ta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 545.0 kB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=38158f70f6ec68c37d6ff2a266f5cd2eb3c1888d20134e589120ce2eda1c65fe
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/quantvesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Mounted at /content/drive


In [3]:
# Import the function from the script
import myProspects_Upside_Analysis_v2

all = 1

# Call the function to get the DataFrame
df_prospects_features = myProspects_Upside_Analysis_v2.get_prospects_features_v2(all)
cols = ['Symbol', 'Close', 'FTT',  'Dev%_200', 'Dev%_PE', 'Spread%', 'Conviction', 'RSI_14', 'RSP', 'Today%', 'FTT%', 'ATH%','Gained%', 'Criteria', 'Strategy', 'Sales_Grwth%', 'Profit_Grwth%', 'Category', 'InFolio', 'MCap', 'MBQ', 'ROE%', 'CFO_2_EBITDA%', 'ROE%/PE']
df_prospects_features = df_prospects_features[cols].drop_duplicates()

Run date time (IST): 2025-07-17 13:49:43
-------------------


ERROR:yfinance:$ISEC.NS: possibly delisted; no price data found  (period=1mo)


get_prospects_features
qualified stocks: 289
with latest results: 84
still star stocks: 62


In [4]:
cols = ['Symbol', 'FTT', 'Dev%_200', 'Dev%_PE', 'Spread%', 'Conviction', 'RSI_14', 'RSP', 'FTT%', 'ATH%','Gained%', 'CumlRnk','ROE%/PE', 'Criteria', 'Strategy', 'Category', 'InFolio']

tmp_df = df_prospects_features
list_x40 = list(tmp_df[tmp_df['MBQ'].str.contains('X40', na=False)]['Symbol'].values)
list_x5k = list(tmp_df[tmp_df['MBQ'].str.contains('X5K', na=False)]['Symbol'].values)
list_x40n = list(tmp_df[tmp_df['MBQ'].str.contains('X40N', na=False)]['Symbol'].values)
list_hc = list(tmp_df[tmp_df['Conviction'].isin(['X-LC', 'H-LC', 'X-MC', 'X-SC'])]['Symbol'].values)
list_ox40 = list(tmp_df[tmp_df['MBQ'].str.contains('OX40', na=False)]['Symbol'].values)
list_ox40n = list(tmp_df[tmp_df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)
#len(set(list_x40 + list_x5k) - set( list_x40n))
stock_list = list(set(list(set(list_x40 + list_x5k + list_hc)- set(list_ox40 + list_ox40n))))

In [5]:
df_tmp = df_prospects_features.copy()

# value ranks (ascending better for deviation from PE and 200DMA)
df_tmp['Rnk_Dev%_200'] = df_tmp['Dev%_200'].rank(method='min', ascending=True)
df_tmp['Rnk_Dev%_PE'] = df_tmp['Dev%_PE'].rank(method='min', ascending=True)
df_tmp['Rnk_ROE%/PE'] = df_tmp['ROE%/PE'].rank(method='max', ascending=False)

# growth ranks
df_tmp['Rnk_Sales_Grwth%'] = df_tmp['Sales_Grwth%'].rank(method='max', ascending=False)
df_tmp['Rnk_Profit_Grwth%'] = df_tmp['Profit_Grwth%'].rank(method='max', ascending=False)

# quality ranks
df_tmp['Rnk_FTT%'] = df_tmp['FTT%'].rank(method='max', ascending=False)
df_tmp['Rnk_MCap'] = df_tmp['MCap'].rank(method='max', ascending=False)
df_tmp['Rnk_ROE%'] = df_tmp['ROE%'].rank(method='max', ascending=False)
df_tmp['Rnk_CFO_2_EBITDA%'] = df_tmp['CFO_2_EBITDA%'].rank(method='max', ascending=False)

# optional Momentum ranks
df_tmp['Rnk_RSI_14'] = df_tmp['RSI_14'].rank(method='max', ascending=False)
df_tmp['Rnk_Gained%'] = df_tmp['Gained%'].rank(method='min', ascending=True)

# category scores
df_tmp['Rank_Value'] = df_tmp[['Rnk_Dev%_200', 'Rnk_Dev%_PE', 'Rnk_ROE%/PE']].sum(axis=1)
df_tmp['Rank_Growth'] = df_tmp[['Rnk_Sales_Grwth%', 'Rnk_Profit_Grwth%']].sum(axis=1)
df_tmp['Rank_Quality'] = df_tmp[['Rnk_FTT%', 'Rnk_MCap', 'Rnk_ROE%', 'Rnk_CFO_2_EBITDA%']].sum(axis=1)
df_tmp['Rank_Momentum'] = df_tmp[['Rnk_Gained%', 'Rnk_RSI_14']].sum(axis=1)

# final cumulative score (you can weigh categories if needed)
df_tmp['Ovrl_Rank'] = df_tmp[['Rank_Value', 'Rank_Growth', 'Rank_Quality', 'Rank_Momentum']].sum(axis=1)

# category-wise rank
df_tmp['Cat_Rank'] = df_tmp.groupby('Conviction')['Ovrl_Rank'].rank(ascending=True, method='min')
priority_map = {'X-LC': 0,'H-LC': 1,'X-MC': 2,'X-SC': 3,'M-LC': 4,'H-MC': 5,'H-SC': 6,'L-LC': 7,'M-MC': 8,'M-SC': 9,'L-MC': 10,'L-SC': 11}
df_tmp['Conviction_Priority'] = df_tmp['Conviction'].map(priority_map)
df_tmp = df_tmp.sort_values(by=['Conviction_Priority', 'Cat_Rank']).reset_index(drop=True)

# final rank
df_tmp['CumlRnk'] = df_tmp.index + 1

# drop intermediate rank columns (optional)
drop_cols = [col for col in df_tmp.columns if col.startswith('Rnk_')]
df_tmp.drop(columns=drop_cols, inplace=True)

# reassign to main dataframe
df_prospects_features = df_tmp[cols]

# display
data_table.DataTable(df_prospects_features.sort_values(by='CumlRnk'), include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
0,VBL,672.28,-9.67,-13.02,14.96,H-LC,63.0,57.99,37.06,37.06,12.70,1,0.4,X40N,ATH,FMCG,1.0
1,ENRIN,3176.70,4.90,NaN,0.34,H-LC,65.0,88.54,3.00,3.00,17.91,2,NaN,AR,ATH,ELECTRICAL,1.0
2,HINDZINC,730.22,-4.35,15.78,3.34,H-LC,43.0,1.74,67.79,67.79,14.56,3,4.1,X5K,ATH,METALS,1.0
3,TCS,4998.00,-14.43,-22.88,10.73,H-LC,30.0,2.78,55.05,38.88,0.98,4,2.2,X40,BTT,IT,1.0
4,ITC,505.32,-1.48,-34.02,3.35,H-LC,62.0,40.97,18.73,18.73,9.77,5,1.8,X40,ATH,FMCG,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,SPORTKING,157.26,33.99,NaN,14.63,NaN,75.0,99.31,16.20,16.20,88.80,287,NaN,AR,ATH,TEXTILES,NaN
287,TASTYBITE,20387.66,8.59,NaN,7.83,NaN,55.0,52.43,82.69,82.69,43.34,288,NaN,XR,ATH,FMCG,NaN
288,V2RETAIL,2031.65,13.95,NaN,14.72,NaN,49.0,19.44,8.71,8.71,147.92,289,NaN,AR,ATH,TEXTILES,NaN
289,VEEDOL,3026.21,3.05,NaN,2.20,NaN,57.0,43.06,78.96,78.96,26.26,290,NaN,XR,ATH,CHEMICALS,NaN


In [6]:
# most interesting sectors
df_tmp = df_prospects_features.groupby('Category').agg(
    Count=('CumlRnk', 'count'),
    Total=('CumlRnk', 'sum'),
    Average=('CumlRnk', 'mean')
).reset_index()
df_tmp['Average'] = round(df_tmp['Average'],2)
df_tmp = df_tmp[df_tmp['Count'] > 3]
df_tmp.sort_values(by='Average')

,Category,Count,Total,Average
27,METALS,5,240,48.00
7,CEMENT,4,347,86.75
23,INSURANCE,8,804,100.50
16,FMCG,20,2075,103.75
12,ELECTRICAL,9,965,107.22
3,AUTO,12,1313,109.42
24,IT,18,2115,117.50
31,PAINTS,5,617,123.40
35,POWER,6,793,132.17
33,PHARMA,17,2297,135.12


In [7]:
# module to print date timestamp
def read_csv(csv_file_name):
  csv_file_path = os.path.join(project_path, 'data', csv_file_name)
  df = pd.read_csv(csv_file_path)
  return df

mypps_df = read_csv('myProspectsScrips.csv')

# Suppose this is your updated DataFrame with new cuml_rnk values
cols1 = ['Symbol', 'CumlRnk']
df_tmp = df_prospects_features[cols1]

# Merge the two DataFrames on 'symbol', keeping the CSV structure
mypps_df = mypps_df.merge(df_tmp[['Symbol', 'CumlRnk']], on='Symbol', how='left', suffixes=('', '_new'))

# Replace 'cuml_rnk' with the new value only if it exists in df
mypps_df['CumlRnk'] = mypps_df['CumlRnk_new'].combine_first(mypps_df['CumlRnk'])

# Drop the helper column
mypps_df.drop(columns=['CumlRnk_new'], inplace=True)

# Save back to CSV
#mypps_df.to_csv(project_path+'/data/myProspectsScrips.csv', index=False)

In [8]:
# High quality (X40 + X40N + X5K + X-LC + H-LC + X-MC + X-SC) prospects
df_tmp = df_prospects_features[df_prospects_features['Symbol'].isin(stock_list)]
df_tmp = df_tmp[(df_tmp['FTT%']>20) & (df_tmp['Dev%_PE']<0)]
data_table.DataTable(df_tmp.sort_values(by = 'CumlRnk', ascending=True), include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
0,VBL,672.28,-9.67,-13.02,14.96,H-LC,63.0,57.99,37.06,37.06,12.70,1,0.4,X40N,ATH,FMCG,1.0
3,TCS,4998.00,-14.43,-22.88,10.73,H-LC,30.0,2.78,55.05,38.88,0.98,4,2.2,X40,BTT,IT,1.0
5,INFY,2275.00,-7.28,-11.34,7.46,H-LC,47.0,12.50,43.14,24.08,15.39,6,1.2,X40,BTT,IT,1.0
6,ABB,7934.00,-10.64,-38.60,6.75,H-LC,35.0,5.90,41.20,59.34,14.55,7,0.5,AR,NTT,ELECTRICAL,1.0
10,CIPLA,1795.00,-0.68,-22.84,0.27,H-LC,46.0,16.67,20.89,13.18,7.23,11,0.8,X40N,BTT,PHARMA,1.0
16,ASIANPAINT,4250.00,-0.54,-17.81,4.23,H-LC,56.0,69.10,76.94,44.28,13.70,17,0.3,X40,BTT,PAINTS,1.0
22,BAJAJHFL,152.00,-1.68,-2.81,2.96,H-LC,55.0,40.62,23.68,47.68,16.23,23,0.3,X40N,BTT,FINANCE,1.0
27,LTIM,7230.20,-3.34,-2.18,3.72,H-LC,47.0,8.68,38.28,38.28,30.04,28,0.6,X200,ATH,IT,1.0
30,SIEMENS,7969.85,-40.33,-8.05,38.55,H-LC,40.0,6.60,156.43,156.43,14.44,31,0.4,AR,ATH,ELECTRICAL,1.0
35,DMART,5201.00,2.58,-20.93,7.15,H-LC,40.0,23.61,28.32,33.02,19.07,36,0.1,X40N,NTT,FMCG,1.0


In [9]:
# x24/x25
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XY24', 'X25'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
196,QUESS,986.00,-42.99,-30.58,44.41,M-SC,51.0,41.67,214.46,281.83,5.15,197,0.9,XY24,NTT,MISC,1.0
195,EASEMYTRIP,26.40,-23.00,9.17,25.16,M-SC,55.0,23.09,144.90,217.25,4.26,196,0.7,XY24,NTT,TRAVEL,1.0
128,BANDHANBNK,400.00,11.98,-28.38,9.99,H-SC,58.0,66.32,116.49,285.89,41.79,129,1.1,XY24,NTT,BANKS,1.0
173,ACC,3906.00,-2.37,-32.87,6.04,M-MC,63.0,71.53,97.28,39.46,9.89,174,0.9,XY24,BTT,CEMENT,1.0
136,REPCOHOME,880.00,8.65,-51.56,4.39,H-SC,65.0,74.31,94.69,86.55,45.22,137,2.2,XY24,NTT,FINANCE,1.0
152,SFL,1287.00,-13.27,33.87,14.00,H-SC,46.0,10.42,87.84,187.85,9.91,153,0.0,XY24,NTT,MISC,1.0
223,GREENPANEL,537.00,-4.22,223.64,13.61,M-SC,71.0,82.99,81.45,102.64,35.85,224,0.2,XY24,NTT,MISC,1.0
233,ATULAUTO,844.00,-10.00,3440.54,12.53,M-SC,56.0,49.31,80.34,70.88,12.81,234,0.0,XY24,NTT,AUTO,1.0
80,AWL,485.00,-2.71,-67.17,7.99,H-MC,64.0,65.28,75.53,204.38,17.20,81,0.5,XY24,NTT,FMCG,1.0
220,STARHEALTH,761.00,-1.72,-3.24,3.99,M-SC,54.0,23.09,74.40,107.83,27.18,221,0.2,XY24,NTT,INSURANCE,1.0


In [10]:
# X-MC and X-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('X-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['X-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio


In [11]:
# H-MC and H-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('H-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['H-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
127,TANLA,1963.11,6.24,-30.49,4.70,H-SC,60.0,11.81,200.47,200.47,56.40,128,1.4,AR,ATH,IT,1.0
132,RELAXO,1176.00,-5.59,-27.18,18.49,H-SC,69.0,98.96,128.55,176.64,28.67,133,0.1,X40N,NTT,FOOTWEAR,1.0
117,ROUTE,2249.26,-14.48,-37.54,16.47,H-SC,54.0,11.11,121.58,121.58,11.26,118,0.7,SR,ATH,IT,1.0
128,BANDHANBNK,400.00,11.98,-28.38,9.99,H-SC,58.0,66.32,116.49,285.89,41.79,129,1.1,XY24,NTT,BANKS,1.0
124,VAIBHAVGBL,521.00,-5.60,58.68,8.03,H-SC,56.0,42.01,113.93,280.90,22.17,125,0.4,XR,NTT,JEWELLERY,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,BOSCHLTD,38795.60,20.02,38.84,8.19,H-MC,83.0,96.88,1.00,1.00,46.70,112,0.3,X40N,ATH,AUTO,NaN
98,MARICO,740.20,10.48,10.86,8.44,H-MC,61.0,74.65,0.75,0.75,25.00,99,0.7,X40,ATH,FMCG,NaN
113,MRF,152470.00,20.19,23.03,14.55,H-MC,68.0,84.38,0.58,0.58,47.67,114,0.3,X40N,ATH,TYRES,NaN
143,GILLETTE,11104.00,22.56,14.39,17.57,H-SC,67.0,72.22,0.00,0.00,50.33,144,0.6,X40,ATH,FMCG,NaN


In [12]:
# x40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
16,ASIANPAINT,4250.00,-0.54,-17.81,4.23,H-LC,56.0,69.10,76.94,44.28,13.70,17,0.3,X40,BTT,PAINTS,1.0
211,BATAINDIA,2096.00,-5.00,-31.44,5.25,M-SC,50.0,36.11,69.72,76.53,4.58,212,0.3,X40,NTT,FOOTWEAR,1.0
3,TCS,4998.00,-14.43,-22.88,10.73,H-LC,30.0,2.78,55.05,38.88,0.98,4,2.2,X40,BTT,IT,1.0
12,BAJAJ-AUTO,12353.32,-3.59,46.14,3.49,H-LC,51.0,30.90,48.33,48.33,16.86,13,0.7,X40,ATH,AUTO,NaN
5,INFY,2275.00,-7.28,-11.34,7.46,H-LC,47.0,12.50,43.14,24.08,15.39,6,1.2,X40,BTT,IT,1.0
40,BAJAJFINSV,2758.00,9.89,-0.44,9.38,H-LC,53.0,35.76,35.89,57.68,32.79,41,0.4,X40,BTT,FINANCE,1.0
87,PGHH,18062.58,-7.70,-20.61,7.88,H-MC,48.0,25.00,35.23,35.23,4.43,88,1.3,X40,ATH,FMCG,1.0
88,HAVELLS,2069.16,-4.68,-7.58,3.64,H-MC,46.0,26.22,35.17,35.17,9.21,89,0.3,X40,ATH,ELECTRICAL,1.0
9,HCLTECH,1972.35,-9.22,6.70,2.36,H-LC,25.0,1.39,27.13,27.13,14.21,10,1.0,X40,ATH,IT,1.0
130,SANOFI,7552.00,0.34,63.60,2.64,H-SC,45.0,17.01,26.31,65.47,22.07,131,1.4,X40,NTT,PHARMA,NaN


In [13]:
# x40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40N'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
132,RELAXO,1176.00,-5.59,-27.18,18.49,H-SC,69.0,98.96,128.55,176.64,28.67,133,0.1,X40N,NTT,FOOTWEAR,1.0
214,TEAMLEASE,3487.00,-15.65,163.03,15.57,M-SC,50.0,13.89,75.76,166.17,17.05,215,0.4,X40N,NTT,MISC,NaN
103,HONAUT,58357.33,3.11,-15.36,3.06,H-MC,62.0,75.69,43.38,43.38,25.00,104,0.2,X40N,ATH,ELECTRICAL,1.0
0,VBL,672.28,-9.67,-13.02,14.96,H-LC,63.0,57.99,37.06,37.06,12.70,1,0.4,X40N,ATH,FMCG,1.0
35,DMART,5201.00,2.58,-20.93,7.15,H-LC,40.0,23.61,28.32,33.02,19.07,36,0.1,X40N,NTT,FMCG,1.0
160,KAJARIACER,1541.71,15.75,33.83,7.91,H-SC,77.0,97.22,26.25,26.25,56.67,161,0.2,X40N,ATH,CERAMICS,NaN
119,BLUESTARCO,2337.55,-0.28,0.40,12.40,H-SC,70.0,88.89,24.16,24.16,22.87,120,0.3,X40N,ATH,AC,1.0
22,BAJAJHFL,152.00,-1.68,-2.81,2.96,H-LC,55.0,40.62,23.68,47.68,16.23,23,0.3,X40N,BTT,FINANCE,1.0
121,CAMS,5226.82,2.27,5.43,3.16,H-SC,56.0,38.19,23.51,23.51,35.66,122,1.0,X40N,ATH,MISC,1.0
10,CIPLA,1795.00,-0.68,-22.84,0.27,H-LC,46.0,16.67,20.89,13.18,7.23,11,0.8,X40N,BTT,PHARMA,1.0


In [14]:
# xr, sr, ar
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XR','SR', 'AR'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
273,BAJAJHIND,442.93,0.11,NaN,8.68,NaN,57.0,93.75,1538.66,1538.66,54.37,274,NaN,XR,ATH,SUGAR,NaN
255,CENTRALBK,189.16,-17.74,-38.15,18.64,L-MC,50.0,43.75,389.29,389.29,14.72,256,1.3,XR,ATH,BANKS,NaN
262,ALOKINDS,79.81,11.55,NaN,4.20,L-SC,66.0,90.28,257.73,257.73,49.93,263,NaN,AR,ATH,TEXTILES,NaN
271,APEX,849.16,5.48,NaN,4.36,NaN,57.0,65.62,244.03,244.03,25.45,272,NaN,AR,ATH,MISC,NaN
217,IGL,707.00,8.10,-3.59,6.46,M-SC,57.0,67.36,220.61,28.57,41.63,218,0.9,AR,BTT,GAS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,JSWSTEEL,1060.34,5.94,169.29,4.95,H-LC,57.0,55.21,1.93,1.93,20.76,49,0.1,AR,ATH,STEEL,NaN
114,SRF,3286.50,19.33,106.31,17.85,H-MC,58.0,57.29,1.85,1.85,50.71,115,0.1,XR,ATH,CHEMICALS,NaN
245,JMFINANCIL,172.18,38.13,54.21,31.72,M-SC,71.0,97.57,1.34,1.34,105.91,246,0.4,XR,ATH,FINANCE,NaN
72,MUTHOOTFIN,2665.40,22.00,41.23,21.14,M-LC,64.0,25.35,0.73,0.73,54.48,73,1.0,XR,ATH,FINANCE,NaN
